In [1]:
##############################################################################
'''
Khan Bank ECL calculation engine
Version number: 0.3
Date: 2024-12-31
Remark: Developing version
'''
##############################################################################

'\nKhan Bank ECL calculation engine\nVersion number: 0.3\nDate: 2024-12-31\nRemark: Developing version\n'

In [2]:
import os
os.chdir("C:\\Users\\EB735DB\\OneDrive - EY\\Desktop\\Workdir\\Credit Risk\\ECL engine\\Khan Bank\\khb_engine\\app")

In [3]:
# Load packages
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict

from input_handler.data_preprocessor import data_preprocessor
from input_handler.load_parameters import (load_configuration_file,
                                           load_parameters)
from input_handler.env_setting import run_setting
from ecl_engine import output_file_handler as ofh

In [6]:

##############################################################################
# ECL engine main function
##############################################################################
configPath = Path(r'C:\Users\EB735DB\OneDrive - EY\Desktop\Workdir\Credit Risk\ECL engine\Khan Bank\khb_engine\run_config_file_kb_2505.json')
c = load_configuration_file(configPath=configPath)
rc = run_setting(run_config=c)

In [11]:
rc.inDataPath

WindowsPath('C:/Users/EB735DB/OneDrive - EY/Desktop/Workdir/Credit Risk/ECL engine/Khan Bank/99_Data_Server_kb/99_Data_Server/production/20250531/data/01_input')

In [13]:
param = load_parameters(parmPath=rc.parmPath)
dp = data_preprocessor(context=rc)
# #instr_df, fx_df, repay_df = dp.load_input_data(param=param)
instr_df_raw, fx_df, repay_df = dp.load_input_data(param=param)

In [15]:
# Filter out required run scope by users
proc_dfs = dp.run(instr_df_raw=instr_df_raw, run_scope=rc.run_scope,param=param)

TypeError: 'float' object cannot be interpreted as an integer

In [7]:
proc_dfs[1].query("CONTRACT_ID == '50000011200000000'")

,REPORT_DATE,OBL_NAME,XTRT_DATE,DATA_SOURCE_CD,CONTRACT_ID,CUST_ID,PRTFLO_ID,INSTR_TYPE,INSTR_SUB_TYPE,IFRS9_MEAS_TYPE,...,LGD_APPROACH,LGD_PRODUCT_DIGIT,LGD_POOL_ID,EAD_POOL_ID,LIFETIME_POOL_ID,PREPAYMENT_POOL_ID,LIFETIME_VAL,MAT_DATE_ADJ,CCF_VAL,EAD_OCY
383645,2024-12-31,nan,2024-12-31,NON_LOAN,50000011200000000,00009511800001197,Cash,Nostro,Current account with BoM,AC,...,NNL,53,NNL53000015,ULF099,XX099,XX099,NaN,2025-01-01,0.0,2.105888e+11


In [7]:
from ecl_engine import collective_assessment as ca
ce = ca.ecl_engine(context=rc)

In [7]:
#test_instr = proc_dfs[1].query("CONTRACT_ID == '50000011200000000'")
ecl_df, ecl_interim_df = ce.run(instr_df=proc_dfs[1],
                                repay_df=repay_df,
                                param=param)

In [8]:
ecl_interim_df.query("CONTRACT_ID == '50000011200000000'")[['mar_PD','cum_days_in_month','PD_POOL_ID','marPD_12m_freq']]

,mar_PD,cum_days_in_month,PD_POOL_ID,marPD_12m_freq
186087,0.0011,0.032258,SL015,0.000035


In [9]:
ecl_df.query("CONTRACT_ID == '50000011200000000'")

,REPORT_DATE,OBL_NAME,XTRT_DATE,DATA_SOURCE_CD,CONTRACT_ID,CUST_ID,PRTFLO_ID,INSTR_TYPE,INSTR_SUB_TYPE,IFRS9_MEAS_TYPE,...,CF_DATE_MAX,is_suff_cf,IFRS9_PD_12M,IFRS9_PD_12M_MADJ,IFRS9_PD_LT,IFRS9_LGD_12M,IFRS9_LGD_LT,ECL_ENGINE_12M_OCY,ECL_ENGINE_LT_OCY,ECL_ENGINE_OCY
671745,2024-12-31,nan,2024-12-31,NON_LOAN,50000011200000000,00009511800001197,Cash,Nostro,Current account with BoM,AC,...,NaT,False,0.0011,0.000035,0.000035,0.6411,0.6411,4.790977e+06,4.790977e+06,4.790977e+06


In [21]:
test_instr = proc_dfs[1].query("CONTRACT_ID == '50000011200000000'")
ecl_df_test, ecl_interim_df_test = ce.run(instr_df=test_instr,
                                repay_df=repay_df,
                                param=param)

In [22]:
ecl_interim_df_test[['mar_PD','cum_days_in_month','PD_POOL_ID','marPD_12m_freq']]

,mar_PD,cum_days_in_month,PD_POOL_ID,marPD_12m_freq
0,0.0011,0.032258,SL015,0.000035


In [24]:
proc_dfs[1].query("CONTRACT_ID == '50000011200000000'").PD_POOL_ID.str[:2].isin(['GC', 'SL', 'SF']).all()

True

In [23]:
test_instr.PD_POOL_ID.str[:2].isin(['GC', 'SL', 'SF']).all()

True

In [11]:
ecl_interim_df.to_clipboard()

In [ ]:
#split

In [9]:
ca_df_valid, ca_df_invalid = ce.filter_cashflow_valid_scope(df=proc_dfs[1],
                                                                      repay_df=repay_df)

In [10]:
df_1 = ce._merge_repayment_schedule(ca_df_valid,repay_df)

In [13]:
ca_df_invalid_on = ca_df_invalid.query(
    "ON_OFF_BAL_IND =='ON' and DRAWN_BAL_OCY > 0 and STAGE_FINAL < 3")
repay_df_on_invalid = ce.repay_df_creation_bycondition(
                ca_df_invalid_on)

In [14]:
repay_df_on_invalid

,MAT_DATE_ADJ,CONTRACT_ID,UNPAID_PRIN_BAL_BEG_OCY,CF_DATE,is_last_row,INT_ACCR
0,2025-12-31,00000005914295252,2.118348e+09,2025-01-31,False,0
1,2025-12-31,00000005914295252,2.118348e+09,2025-02-28,False,0
2,2025-12-31,00000005914295252,2.118348e+09,2025-03-31,False,0
3,2025-12-31,00000005914295252,2.118348e+09,2025-04-30,False,0
4,2025-12-31,00000005914295252,2.118348e+09,2025-05-31,False,0
...,...,...,...,...,...,...
187028,2025-12-31,00000005222052531,1.518620e+03,2025-08-31,False,0
187029,2025-12-31,00000005222052531,1.518620e+03,2025-09-30,False,0
187030,2025-12-31,00000005222052531,1.518620e+03,2025-10-31,False,0
187031,2025-12-31,00000005222052531,1.518620e+03,2025-11-30,False,0


In [12]:
repay_df['UNPAID_PRIN_BAL_BEG_OCY']

0           2.470575e+07
1           6.021343e+06
2           8.531713e+06
3           8.251631e+07
4           3.610448e+06
                ...     
12248478    3.293679e+07
12248479    1.105235e+07
12248480    2.029374e+07
12248481    1.822762e+07
12248482    2.514142e+06
Name: UNPAID_PRIN_BAL_BEG_OCY, Length: 12248483, dtype: float64

In [11]:
df_1.head()

,REPORT_DATE,CONTRACT_ID,ON_OFF_BAL_IND,STAGE_FINAL,EFF_INT_RT,MAT_DATE,MAT_DATE_ADJ,PD_POOL_ID,LGD_POOL_ID,PREPAYMENT_POOL_ID,PENALTY_OCY,OTHER_FEE_AND_CHARGES_OCY,ACRU_INT_OCY,CF_DATE,EAD_UNADJ
704720,2024-12-31,00000005000050624,ON,1,0.06,2025-02-28,2025-02-28,CS001,COL66000XXX,LL066,0.0,0.0,1368.55637,2025-01-31,4.053004e+05
704721,2024-12-31,00000005000050624,ON,1,0.06,2025-02-28,2025-02-28,CS001,COL66000XXX,LL066,0.0,0.0,1368.55637,2025-02-28,3.958007e+05
3462602,2024-12-31,00000005000052837,ON,1,0.06,2025-06-30,2025-06-30,CS001,COL66000XXX,LL066,0.0,0.0,825.28061,2025-01-31,1.793605e+06
3462605,2024-12-31,00000005000052837,ON,1,0.06,2025-06-30,2025-06-30,CS001,COL66000XXX,LL066,0.0,0.0,825.28061,2025-02-28,1.482688e+06
3462601,2024-12-31,00000005000052837,ON,1,0.06,2025-06-30,2025-06-30,CS001,COL66000XXX,LL066,0.0,0.0,825.28061,2025-03-31,1.171621e+06


In [ ]:
# #solve dup id
# # Define the ID to replace and the new values
# id_to_replace = '00000005038159245'
# new_id_first = '00000005038159245A'
# new_id_second = '00000005038159245B'

# # Find indices of occurrences where ON_OFF_BAL_IND is 'OFF'
# matching_indices = df[(df['CONTRACT_ID'] == id_to_replace) & (df['ON_OFF_BAL_IND'] == 'OFF')].index

# # Replace the first occurrence
# if len(matching_indices) > 0:
#     df.at[matching_indices[0], 'CONTRACT_ID'] = new_id_first

# # Replace the second occurrence
# if len(matching_indices) > 1:
#     df.at[matching_indices[1], 'CONTRACT_ID'] = new_id_second
# import csv
# # Ensure Contract_ID column is of type string
# df['CONTRACT_ID'] = df['CONTRACT_ID'].astype(str)
# df['CUST_ID'] = df['CUST_ID'].astype(str)
# # Save to CSV
# df.to_csv('./working/instrument_table_fixeddupid.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [6]:
# Filter out required run scope by users
proc_dfs = dp.run(instr_df_raw=instr_df_raw, run_scope=rc.run_scope,param=param)

In [13]:
from ecl_engine import collective_assessment as ca
ce = ca.ecl_engine(context=rc)

In [14]:
instr_df = proc_dfs[1]
instr_df.columns

Index(['REPORT_DATE', 'OBL_NAME', 'XTRT_DATE', 'DATA_SOURCE_CD', 'CONTRACT_ID',
       'CUST_ID', 'PRTFLO_ID', 'INSTR_TYPE', 'INSTR_SUB_TYPE',
       'IFRS9_MEAS_TYPE',
       ...
       'LGD_APPROACH', 'LGD_PRODUCT_DIGIT', 'LGD_POOL_ID', 'EAD_POOL_ID',
       'LIFETIME_POOL_ID', 'PREPAYMENT_POOL_ID', 'LIFETIME_VAL',
       'MAT_DATE_ADJ', 'CCF_VAL', 'EAD_OCY'],
      dtype='object', length=106)

In [15]:
ecl_df, ecl_interim_df = ce.run(instr_df=proc_dfs[1],
                                repay_df=repay_df,
                                param=param)

In [10]:
ecl_interim_df.ON_OFF_BAL_IND.unique()

array(['ON', 'OFF'], dtype=object)

In [16]:
from ecl_engine import management_overlay as mo
me = mo.overlay_engine(context=rc)
ecl_df_final = me.run(ecl_df=ecl_df, param=param)

ecl_df_fmt, ecl_interim_df_fmt = ofh.run(df=ecl_df_final,
                                            calc_proc_df=ecl_interim_df,
                                            fx_df=fx_df,
                                            param=param,
                                            dtype_tbl=rc.dtype_tbl,
                                            calc_file_partition=20,
                                            resultPath=rc.resultPath,
                                            is_export=True)

Filename: C:\Users\EB735DB\OneDrive - EY\Desktop\Workdir\Credit Risk\ECL engine\Khan Bank\khb_engine\app\ecl_engine\management_overlay.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   102   4322.7 MiB   4322.7 MiB           1       @profile
   103                                             def run(self,
   104                                                     ecl_df: pd.DataFrame,
   105                                                     param: Dict[str, pd.DataFrame]
   106                                                     ) -> pd.DataFrame:
   107                                         
   108   4325.3 MiB      2.6 MiB           1           overlay_param = self.extract_overlay_info(param=param)
   109   5625.3 MiB   1300.0 MiB           2           ecl_df_final = self.calculate_overlay_ecl(ecl_df,
   110   4325.3 MiB      0.0 MiB           1                                                     overlay_param=overlay_param)
   111                              

In [11]:
del(pm)
del(prm)

In [12]:
from ecl_engine import proxy_model as pm
prm = pm.ProxyModel(context=rc)

In [8]:
px_df_1 = prm.get_proxy_ecl_key(px_df_=proc_dfs[3], param=param)

In [14]:
px_ecl_pct_assign = param['ProxyECL']
px_ecl_pct_assign['PROXY_ECL_KEY'] = px_ecl_pct_assign['PORTFOLIO_CODE'] + \
            "-" + px_ecl_pct_assign['STAGE_FINAL'].astype(int).astype(str)
px_ecl_pct = px_ecl_pct_assign[['PROXY_ECL_KEY', 'PROXY_ECL_PCT']]

In [15]:
px_ecl_pct

,PROXY_ECL_KEY,PROXY_ECL_PCT
3,KB91-1,0.001
4,KB91-2,0.050
5,KB91-3,0.700
6,KB92-1,0.001
7,KB92-2,0.050
8,KB92-3,0.700


In [ ]:
proxy_result = pd.merge(px_df_1, px_ecl_pct,
                                how='left', on='PROXY_ECL_KEY')

In [29]:
proxy_result

,REPORT_DATE,OBL_NAME,XTRT_DATE,DATA_SOURCE_CD,CONTRACT_ID,CUST_ID,PRTFLO_ID,INSTR_TYPE,INSTR_SUB_TYPE,IFRS9_MEAS_TYPE,...,EAD_OCY,Parameter name,PROXY_PORTFOLIO_ID,PROXY_PORTFOLIO_CODE,PROXY_ECL_KEY,CCY_CD_OCY,CCY_CD_RPT,FX_RT,EAD_LCY,PROXY_ECL_PCT
0,2024-12-31,nan,2024-12-31,LOAN,00000005038158582,00009503800007698,1601-3000,factoring,nan,AC,...,36000000.0,7,Factoring,KB92,KB92-1,MNT,MNT,1.0,36000000.0,0.001
1,2024-12-31,nan,2024-12-31,LOAN,00000005038159030,00009503800007698,1601-3000,factoring,nan,AC,...,99000000.0,7,Factoring,KB92,KB92-1,MNT,MNT,1.0,99000000.0,0.001
2,2024-12-31,nan,2024-12-31,LOAN,00000005038158242,00009503800007698,1601-3000,factoring,nan,AC,...,36000000.0,7,Factoring,KB92,KB92-1,MNT,MNT,1.0,36000000.0,0.001
3,2024-12-31,nan,2024-12-31,LOAN,00000005038159109,00009503800007698,1601-3000,factoring,nan,AC,...,148500000.0,7,Factoring,KB92,KB92-1,MNT,MNT,1.0,148500000.0,0.001
4,2024-12-31,nan,2024-12-31,LOAN,00000005038159201,00009503800007698,1601-3000,factoring,nan,AC,...,72000000.0,7,Factoring,KB92,KB92-1,MNT,MNT,1.0,72000000.0,0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561,2024-12-31,nan,2024-12-31,LOAN,ETE2303074668174,00009500401913905,TENDER_GUARANTEE,Performance guarantee issued,nan,AC,...,400000.0,1,Performance guarantee issued,KB91,KB91-1,MNT,MNT,1.0,400000.0,0.001
3562,2024-12-31,nan,2024-12-31,LOAN,PFG2410176592827,00009511100001251,PERFORMANCE_GUARANTEE,Performance guarantee issued,nan,AC,...,15290022.0,2,Performance guarantee issued,KB91,KB91-1,MNT,MNT,1.0,15290022.0,0.001
3563,2024-12-31,nan,2024-12-31,LOAN,ETE2022042508207,00009500400401806,TENDER_GUARANTEE,Performance guarantee issued,nan,AC,...,134316.0,1,Performance guarantee issued,KB91,KB91-1,MNT,MNT,1.0,134316.0,0.001
3564,2024-12-31,nan,2024-12-31,LOAN,WRG2409110004941,00009515600019943,QUALITY_GUARANTEE,Performance guarantee issued,nan,AC,...,1450000.0,3,Performance guarantee issued,KB91,KB91-1,MNT,MNT,1.0,1450000.0,0.001


In [30]:
mask_guarantee = (
    (proxy_result.PROXY_PORTFOLIO_ID.astype(str).str.upper() == 'PERFORMANCE GUARANTEE ISSUED') |
    (proxy_result.PROXY_PORTFOLIO_ID.astype(str).str.upper() == 'FINANCIAL GUARANTEE ISSUED')
)

In [33]:
proxy_result.loc[mask_guarantee, 'EAD_OCY'] = proxy_result.loc[mask_guarantee, 'EAD_OCY'] - proxy_result.loc[mask_guarantee, 'CASH']

In [34]:
proxy_result[mask_guarantee][['EAD_LCY','EAD_OCY','CASH']]

,EAD_LCY,EAD_OCY,CASH
17,896510.0,896510.0,0.0
18,650000.0,0.0,650000.0
19,1750000.0,0.0,1750000.0
20,4000000.0,0.0,4000000.0
21,599965.0,0.0,599965.0
...,...,...,...
3561,400000.0,0.0,400000.0
3562,15290022.0,0.0,15290022.0
3563,134316.0,0.0,134316.0
3564,1450000.0,0.0,1450000.0


In [9]:
px_df_2 = ofh.convert_to_LCY(df=px_df_1, fx_tbl=fx_df, param=param)

In [16]:
proxy_result = prm.get_proxy_result(
            ecl_result=ecl_df_fmt, px_df_1=px_df_1, fx_df=fx_df, param=param)

In [12]:
need_cf_cond = (instr_df.STAGE_FINAL < 3)

ca_cf_input = instr_df[need_cf_cond] # need
ca_nocf_input = instr_df[~need_cf_cond] # # 3. STAGE 3 ECL = EAD*LGD, no interim

In [13]:
# Split the collective assessment run into 2 routes
# One with complete cash flow data
# The other without valid cash flow data     
ca_df_valid, ca_df_invalid = ce.filter_cashflow_valid_scope(df=ca_cf_input,
                                                                repay_df=repay_df)

In [16]:
ca_df_valid.ON_OFF_BAL_IND.unique()

array(['ON'], dtype=object)

In [18]:
# Collective assessment (Valid cash flow route)
# EAD model
if ca_df_valid.shape[0] > 0:
    ecl_df_valid= (ca_df_valid
                                        .pipe(ce._merge_repayment_schedule,
                                                repay_df=repay_df)
                                        .pipe(ce._calculate_days_months)
                                        .pipe(ce._adjust_prepayment, param=param)
                                        .pipe(ce._adjust_ead_valid)
                                        .pipe(ce.calculate_PD_new, param=param)
                                        .pipe(ce.calculate_LGD, param=param)
                                        .pipe(ce.calculate_discount_factor))
                                        # .pipe(ce.calculate_engine_ecl,
                                        #         instr_df=ca_df_valid)
                                        # ), ecl_interim_df_valid 

In [20]:
ca_df_valid

,REPORT_DATE,XTRT_DATE,DATA_SOURCE_CD,CONTRACT_ID,CUST_ID,OBL_NAME,PRTFLO_ID,INSTR_TYPE,INSTR_SUB_TYPE,IFRS9_MEAS_TYPE,...,LIFETIME_VAL,MAT_DATE_ADJ,CCF_VAL,EAD_OCY,KEY,SUM_REPAY_BAL_OCY,is_valid_cashflow,is_in_repay_df,CF_DATE_MAX,is_suff_cf
187,2024-06-30,2024-06-30,LOAN,4034,ADSSKISA,nan,2300-0037,consumer,nan,AC,...,NaN,2025-02-28,0.0,1.363953e+06,4034ON,6.441123e+06,True,True,2025-02-28,True
188,2024-06-30,2024-06-30,LOAN,4255,ADSSEZXW,nan,1500-0001,consumer,nan,AC,...,NaN,2025-06-30,0.0,3.636522e+06,4255ON,2.331331e+07,True,True,2025-06-30,True
189,2024-06-30,2024-06-30,LOAN,4435,ADSTHAOF,nan,1500-0001,consumer,nan,AC,...,NaN,2026-06-30,0.0,1.830368e+07,4435ON,2.293743e+08,True,True,2026-06-30,True
190,2024-06-30,2024-06-30,LOAN,4517,ADSPIRXM,nan,1500-0001,consumer,nan,AC,...,NaN,2028-08-31,0.0,4.215216e+07,4517ON,1.097610e+09,True,True,2028-08-31,True
191,2024-06-30,2024-06-30,LOAN,4522,ADSOMBNW,nan,1500-0001,consumer,nan,AC,...,NaN,2027-01-31,0.0,2.582480e+07,4522ON,4.189142e+08,True,True,2027-01-31,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669023,2024-06-30,2024-06-30,LOAN,8159399,DNGRVDACO,nan,2000-0001,business,nan,AC,...,NaN,2026-01-31,0.0,2.320982e+09,8159399ON,2.363614e+10,True,True,2026-01-31,True
673414,2024-06-30,2024-06-30,LOAN,15131607,DNZXTPWPY,nan,2000-0011,business,nan,AC,...,NaN,2026-05-31,0.0,2.025434e+09,15131607ON,2.973759e+10,True,True,2026-05-31,True
673458,2024-06-30,2024-06-30,LOAN,15195508,DNZXTPWTE,nan,2000-0011,business,nan,AC,...,NaN,2026-07-31,0.0,1.719996e+09,15195508ON,3.374687e+10,True,True,2026-07-31,True
674642,2024-06-30,2024-06-30,LOAN,16121752,DNGJKNNKO,nan,2000-0001,business,nan,AC,...,NaN,2031-06-30,0.0,1.804932e+09,16121752ON,9.470931e+10,True,True,2031-06-30,True


In [ ]:
repay_df.columns

In [12]:
from ecl_engine import collective_assessment as ca
ce = ca.ecl_engine(context=rc)

ecl_df, ecl_interim_df = ce.run(instr_df=proc_dfs[1],
                                repay_df=repay_df,
                                param=param)

In [ ]:
# 250312 test for stage consistency
stage_diff = pd.read_excel(r'C:\Users\EB735DB\Downloads\Test_sample.xlsx',sheet_name='Staging',skiprows=6)

In [ ]:
stage_diff

In [ ]:
stage_diff = stage_diff.query("difference>0")
stage_diff["CONTRACT_ID"] = stage_diff["CONTRACT_ID"].astype(str)

In [ ]:
cases_374 = stage_diff[['Dans','CONTRACT_ID','CUST_ID','KB Stage','STAGE_FINAL - EY','Cross stage','difference']]
cases_374['Dans'] = cases_374['Dans'].astype(str)

In [ ]:
for id_x in cases_374["CUST_ID"]:
    contract_id = cases_374[cases_374["CUST_ID"] == id_x]['CONTRACT_ID']
    df_cust = instr_df.query("CUST_ID == @id_x")
    ey_stage_check = df_cust.query("ON_OFF_BAL_IND == 'ON'")["STAGE_FAC"].max().astype(float)
    kb_stage_check = cases_374.query("CONTRACT_ID == @contract_id")["Cross stage"].values.astype(float)
    if ey_stage_check == kb_stage_check:
        print("equal")
    else:
        print(id_x)

In [ ]:
id_x = "5192331843"
cust_id = cases_374[cases_374["Dans"] == id_x]['CUST_ID'].values[0]
cust_id

In [ ]:
df_cust = instr_df.query("CUST_ID == @cust_id")
ey_stage_check = df_cust.query("ON_OFF_BAL_IND == 'ON'")["STAGE_FAC"].max().astype(float)
ey_stage_check

In [ ]:
df_cust.to_clipboard()

In [ ]:
kb_stage_check = cases_374[cases_374["Dans"] == id_x]['Cross stage'].values[0]
kb_stage_check

In [ ]:
# Initialize lists to store ey_stage_check and equality results
ey_stage_checks = []
equality_results = []

# Iterate over each CONTRACT_ID in cases_374
for id_x in cases_374["Dans"]:
    cust_id = cases_374[cases_374["Dans"] == id_x]['CUST_ID'].values[0]
    # Filter instr_df for the current CUST_ID
    df_cust = instr_df.query("CUST_ID == @cust_id")
    
    # Calculate the maximum STAGE_FAC for ON_OFF_BAL_IND == 'ON'
    ey_stage_check = df_cust[df_cust["ON_OFF_BAL_IND"] == 'ON']["STAGE_FAC"].max()
    
    # Get the corresponding Cross stage value from cases_374
    kb_stage_check = cases_374.loc[cases_374["Dans"] == id_x, "Cross stage"].values[0]
    
    # Append ey_stage_check and equality result to the lists
    ey_stage_checks.append(ey_stage_check)
    equality_results.append("equal" if ey_stage_check == kb_stage_check else "not equal")

# Add the new columns to cases_374
cases_374["ey_stage_check"] = ey_stage_checks
cases_374["equality"] = equality_results

# Display the updated DataFrame
cases_374.to_csv('./check_stage_consistency.csv')

In [ ]:
# 250304 test for bond pd break down
df_bond_case = ca_df_invalid.query("ON_OFF_BAL_IND == 'ON' and DATA_SOURCE_CD == 'NON_LOAN' and STAGE_FINAL == 1 and CONTRACT_ID == 'MN0CBB041682'") #  and CONTRACT_ID == 'MN00BID30344' undra case:MN0CBB041682

In [ ]:
df_bond_case.to_clipboard()

In [ ]:
# # Invalid cash flow route
# # 241112: we seperate on-bal and off-bal calculation due to duplicate keys in credit cards and credit lines
# # 1. ON
ca_df_invalid_on = df_bond_case.query(
    "ON_OFF_BAL_IND =='ON' and DRAWN_BAL_OCY > 0 and STAGE_FINAL < 3")

if ca_df_invalid_on.shape[0] > 0:
    repay_df_on_invalid = ce.repay_df_creation_bycondition(
        ca_df_invalid_on)

    ecl_df_invalid_on, ecl_df_interim_invalid_on = (ca_df_invalid_on
                                                    .pipe(ce._merge_repayment_schedule, repay_df=repay_df_on_invalid)
                                                    .pipe(ce._calculate_days_months)
                                                    .pipe(ce._adjust_prepayment, param=param)
                                                    .pipe(ce.calculate_PD_new, param=param)
                                                    .pipe(ce.calcualte_LGD, param=param)
                                                    .pipe(ce.calculate_discount_factor)
                                                    .pipe(ce.calculate_engine_ecl, instr_df=ca_df_invalid_on))
else:
    ecl_df_invalid_on = pd.DataFrame()
    ecl_df_interim_invalid_on = pd.DataFrame()

In [ ]:
ecl_df_invalid_on_test= (ca_df_invalid_on
                                                    .pipe(ce._merge_repayment_schedule, repay_df=repay_df_on_invalid)
                                                    .pipe(ce._calculate_days_months)
                                                    .pipe(ce._adjust_prepayment, param=param)
                                                    .pipe(ce.calculate_PD_new, param=param)
                                                    .pipe(ce.calcualte_LGD, param=param)
                                                    .pipe(ce.calculate_discount_factor))

In [ ]:
ecl_df = ecl_df_invalid_on_test.copy()

In [ ]:
ecl_df_ = (ecl_df.assign(

            # For Model validation purpose
            IFRS9_PD_12M=(ecl_df.mar_PD *
                          ecl_df.filter_12m
                          ),

            IFRS9_PD_12M_MADJ=(ecl_df.marPD_12m_freq *
                               ecl_df.filter_12m
                               ),

            IFRS9_PD_LT=(ecl_df.marPD_lt_freq),

            IFRS9_LGD_12M=(ecl_df.LGD *
                           ecl_df.filter_12m),

            IFRS9_LGD_LT=(ecl_df.LGD),

            ECL_ENGINE_12M_OCY=(ecl_df.EAD_PPADJ_12M
                                * ecl_df.marPD_12m_freq
                                * ecl_df.LGD
                                * ecl_df.filter_12m
                                * ecl_df.filter_mature
                                * ecl_df.disc_factor),

            ECL_ENGINE_LT_OCY=(ecl_df.EAD_PPADJ_LT
                               * ecl_df.marPD_lt_freq
                               * ecl_df.LGD
                               * ecl_df.filter_mature
                               * ecl_df.disc_factor),
        ))

In [ ]:
ecl_df_agg = (ecl_df_.groupby(by=['CONTRACT_ID'])
                .agg(
    IFRS9_PD_12M=pd.NamedAgg('IFRS9_PD_12M', 'max'),
    IFRS9_PD_12M_MADJ=pd.NamedAgg(
        'IFRS9_PD_12M_MADJ', 'sum'),
    
    IFRS9_PD_LT=pd.NamedAgg('IFRS9_PD_LT', 'sum'),

    IFRS9_LGD_12M=pd.NamedAgg('IFRS9_LGD_12M', 'max'),
    IFRS9_LGD_LT=pd.NamedAgg('IFRS9_LGD_LT', 'mean'),

    ECL_ENGINE_12M_OCY=pd.NamedAgg(
        'ECL_ENGINE_12M_OCY', 'sum'),
    ECL_ENGINE_LT_OCY=pd.NamedAgg('ECL_ENGINE_LT_OCY', 'sum'),)
    .reset_index())

In [ ]:
ecl_df_agg.to_clipboard()

In [ ]:
ecl_df_invalid_on_test['CF_DATE'].dt.day

In [ ]:
ecl_df_invalid_on_test['CF_DATE'].dt.day/ecl_df_invalid_on_test.CF_DATE.dt.days_in_month.astype(int) 

In [ ]:
ecl_df_invalid_on_test.to_clipboard()

In [ ]:
ecl_df_invalid_on.to_clipboard()

In [ ]:
pd_param_bond_mef = ce._apply_mef(
    param_mef=param['MEFmultipliers'], pd_param_bond=param['PD_term_structure'])
pd_param_bond_mef_monthly = ce._convert_bond_pd_param_monthly(pd_param_bond_mef=pd_param_bond_mef)
pd_param_consol = ce._consol_pd_param(
param_bond=pd_param_bond_mef_monthly, param=param)
pd_t = ce._transpose_pd(pd_param=pd_param_consol)

In [ ]:
ead_df_ = (ecl_df_invalid_on_check.merge(
    pd_t, how='left',
    left_on=['PD_POOL_ID', 'months'],
    right_on=['PD_POOL_ID', 'months']))


In [ ]:
ead_df_ = (ead_df_.assign(
            marPD_12m_freq=(ead_df_['mar_PD']*ead_df_['filter_12m']* ead_df_.cum_days_in_month)
                            if ead_df_.PD_POOL_ID.str[:2].isin(['GC', 'SL', 'SF']).all()
                            else ead_df_['mar_PD']*ead_df_['filter_12m']
        ,
            marPD_lt_freq=(ead_df_['mar_PD']* ead_df_.cum_days_in_month)
                            if ead_df_.PD_POOL_ID.str[:2].isin(['GC', 'SL', 'SF']).all()
                            else ead_df_['mar_PD'],
        ))

In [ ]:
ead_df_

In [ ]:
ead_df_.PD_POOL_ID.str[:2].isin(['GC', 'SL', 'SF']).all()

In [ ]:
marPD_12m_freq=(ead_df_['mar_PD']*ead_df_['filter_12m']* ead_df_.cum_days_in_month if ead_df_.PD_POOL_ID.str[:2].isin(['GC', 'SL', 'SF']).all() else ead_df_['mar_PD']*ead_df_['filter_12m'])

In [ ]:
marPD_12m_freq

In [ ]:
ead_df_ = (ead_df_.assign(
    marPD_12m_freq=(ead_df_['mar_PD']*ead_df_['filter_12m']* ead_df_.cum_days_in_month)
                    if ead_df_['PD_POOL_ID'][:2].isin(['GC', 'SL', 'SF'])
                    else ead_df_['mar_PD']*ead_df_['filter_12m']
,
    marPD_lt_freq=(ead_df_['mar_PD']* ead_df_.cum_days_in_month)
                    if ead_df_['PD_POOL_ID'][:2].isin(['GC', 'SL', 'SF'])
                    else ead_df_['mar_PD'],
))

In [ ]:
ecl_df_invalid_on_check.to_clipboard()

In [ ]:
cum_days = ecl_df_invalid_on_check['CF_DATE'].dt.day/ecl_df_invalid_on_check.CF_DATE.dt.days_in_month.astype(int)

In [ ]:
cum_days

In [ ]:
# 250218 filter cases for prototype
# case 1: stage 1 valid on-bal loan (all valid is on-bal)
# case 2: stage 1 invalid on-bal loan
# case 3: stage 2 invalid off-bal loan
# case 4: stage 1 invalid on-bal bond
# case 5: stage 3 invalid on-bal laon
# case 6: stage 3
df_pr_case_1 = ca_df_valid.query("ON_OFF_BAL_IND == 'ON' and DATA_SOURCE_CD == 'LOAN' and STAGE_FINAL == 1 and CONTRACT_ID == '4034'")
df_pr_case_2 = ca_df_invalid.query("ON_OFF_BAL_IND == 'ON' and DATA_SOURCE_CD == 'LOAN' and STAGE_FINAL == 1 and CONTRACT_ID == '138624'")
df_pr_case_3 = ca_df_invalid.query("ON_OFF_BAL_IND == 'OFF' and DATA_SOURCE_CD == 'LOAN' and STAGE_FINAL == 2 and CONTRACT_ID == '21241513'")
df_pr_case_4 = ca_df_invalid.query("ON_OFF_BAL_IND == 'ON' and DATA_SOURCE_CD == 'NON_LOAN' and STAGE_FINAL == 1 and CONTRACT_ID == 'MNBIDD290019'")
df_pr_case_5 = ca_nocf_input.query("ON_OFF_BAL_IND == 'ON' and DATA_SOURCE_CD == 'LOAN' and STAGE_FINAL == 3 and CONTRACT_ID == '21299325'")
df_pr_case_6 = ca_nocf_input.query("ON_OFF_BAL_IND == 'OFF' and STAGE_FINAL == 3 and CONTRACT_ID =='21094793'")

In [ ]:
contract_id_cases = ['4034','138624','21241513','MNBIDD290019','21299325','21094793']
df_pr_cases = instr_df.query("CONTRACT_ID in @contract_id_cases")

In [ ]:
df_pr_cases_valid = df_pr_case_1.copy()
df_pr_cases_invalid = pd.concat([df_pr_case_2,df_pr_case_3,df_pr_case_4])
df_pr_cases_stage3 = pd.concat([df_pr_case_5,df_pr_case_6])

In [ ]:
df_pr_cases_invalid.to_clipboard()

In [ ]:
repay_df_pr_cases = repay_df.query("CONTRACT_ID in @contract_id_cases")

In [ ]:
# Collective assessment (Valid cash flow route)
# EAD model
if df_pr_cases_valid.shape[0] > 0:
    ecl_df_valid, ecl_interim_df_valid = (df_pr_cases_valid
                                        .pipe(ce._merge_repayment_schedule,
                                                repay_df=repay_df)
                                        .pipe(ce._calculate_days_months)
                                        .pipe(ce._adjust_prepayment, param=param)
                                        .pipe(ce._adjust_ead_valid)
                                        .pipe(ce.calculate_PD_new, param=param)
                                        .pipe(ce.calcualte_LGD, param=param)
                                        .pipe(ce.calculate_discount_factor)
                                        .pipe(ce.calculate_engine_ecl,
                                                instr_df=df_pr_cases_valid)
                                        )
else:
    ecl_df_valid = pd.DataFrame()
    ecl_interim_df_valid = pd.DataFrame()

In [ ]:
# # Invalid cash flow route
# # 241112: we seperate on-bal and off-bal calculation due to duplicate keys in credit cards and credit lines
# # 1. ON
ca_df_invalid_on = df_pr_cases_invalid.query(
    "ON_OFF_BAL_IND =='ON' and DRAWN_BAL_OCY > 0 and STAGE_FINAL < 3")

if ca_df_invalid_on.shape[0] > 0:
    repay_df_on_invalid = ce.repay_df_creation_bycondition(
        ca_df_invalid_on)

    ecl_df_invalid_on, ecl_df_interim_invalid_on = (ca_df_invalid_on
                                                    .pipe(ce._merge_repayment_schedule, repay_df=repay_df_on_invalid)
                                                    .pipe(ce._calculate_days_months)
                                                    .pipe(ce._adjust_prepayment, param=param)
                                                    .pipe(ce.calculate_PD_new, param=param)
                                                    .pipe(ce.calcualte_LGD, param=param)
                                                    .pipe(ce.calculate_discount_factor)
                                                    .pipe(ce.calculate_engine_ecl, instr_df=ca_df_invalid_on))
else:
    ecl_df_invalid_on = pd.DataFrame()
    ecl_df_interim_invalid_on = pd.DataFrame()

# # 2. OFF
ca_df_invalid_off = df_pr_cases_invalid.query(
    "ON_OFF_BAL_IND == 'OFF' and UNDRAWN_BAL_OCY > 0 and STAGE_FINAL < 3")
if ca_df_invalid_off.shape[0] > 0:
    repay_df_off_invalid = ce.repay_df_creation_bycondition(
        ca_df_invalid_off)

    ecl_df_invalid_off, ecl_df_interim_invalid_off = (ca_df_invalid_off
                                                    .pipe(ce._merge_repayment_schedule, repay_df=repay_df_off_invalid)
                                                    .pipe(ce._calculate_days_months)
                                                    .pipe(ce._adjust_prepayment, param=param)
                                                    .pipe(ce.calculate_PD_new, param=param)
                                                    .pipe(ce.calcualte_LGD, param=param)
                                                    .pipe(ce.calculate_discount_factor)
                                                    .pipe(ce.calculate_engine_ecl, instr_df=ca_df_invalid_off))
else:
    ecl_df_invalid_off = pd.DataFrame()
    ecl_df_interim_invalid_off = pd.DataFrame()

In [ ]:
repay_df_off_invalid.to_clipboard()

In [ ]:
## TO add monthly index for PD
df_valid_test = ca_df_valid[0:100]

In [ ]:
mapPD_df_valid_test, test_interim = (df_valid_test
                                    .pipe(ce._merge_repayment_schedule,
                                                        repay_df=repay_df)
                                    .pipe(ce._calculate_days_months)
                                    .pipe(ce._adjust_prepayment, param=param)
                                    .pipe(ce._adjust_ead_valid)
                                    .pipe(ce.calculate_PD_new,param=param)
                                    .pipe(ce.calcualte_LGD, param=param)
                                    .pipe(ce.calculate_discount_factor)
                                    .pipe(ce.calculate_engine_ecl,
                                        instr_df=df_valid_test)
                                )
#

In [ ]:
mapPD_df_valid_test.query("CONTRACT_ID == '10066'").to_clipboard()

In [ ]:
test_interim.to_clipboard()

In [ ]:
# invalid test

df_invalid_test = ca_df_invalid.query("INSTR_TYPE == 'Bond' and ON_OFF_BAL_IND =='ON' and DRAWN_BAL_OCY > 0 and STAGE_FINAL < 3 ")

In [ ]:
if df_invalid_test.shape[0] > 0:
        repay_df_on_invalid = ce.repay_df_creation_bycondition(
            df_invalid_test)

        ecl_df_invalid_on, ecl_df_interim_invalid_on = (df_invalid_test
                                                        .pipe(ce._merge_repayment_schedule, repay_df=repay_df_on_invalid)
                                                        .pipe(ce._calculate_days_months)
                                                        .pipe(ce._adjust_prepayment, param=param)
                                                        .pipe(ce.calculate_PD_new, param=param)
                                                        .pipe(ce.calcualte_LGD, param=param)
                                                        .pipe(ce.calculate_discount_factor)
                                                        .pipe(ce.calculate_engine_ecl, instr_df=df_invalid_test))
else:
    ecl_df_invalid_on = pd.DataFrame()
    ecl_df_interim_invalid_on = pd.DataFrame()

In [ ]:
ecl_df_invalid_on.query("CONTRACT_ID == '20005000643'").to_clipboard()

In [ ]:
pd_param_bond_mef = ce._apply_mef(
    param_mef=param['MEFmultipliers'], pd_param_bond=param['PD_term_structure'])
pd_param_bond_mef_monthly = ce._convert_bond_pd_param_monthly(pd_param_bond_mef=pd_param_bond_mef)
pd_param_consol = ce._consol_pd_param(
    param_bond=pd_param_bond_mef_monthly, param=param)
pd_t = ce._transpose_pd(pd_param=pd_param_consol)

In [ ]:
mapPD_df_valid_test.to_clipboard()

In [ ]:
###################

In [ ]:
pd_param_bond_mef = ce._apply_mef(
    param_mef=param['MEFmultipliers'], pd_param_bond=param['PD_term_structure'])
pd_param_bond_mef_monthly = ce._convert_bond_pd_param_monthly(pd_param_bond_mef=pd_param_bond_mef)

In [ ]:
pd_param_consol = ce._consol_pd_param(
    param_bond=pd_param_bond_mef_monthly, param=param)

In [ ]:
pd_param_consol

In [ ]:
pd_t = ce._transpose_pd(pd_param=pd_param_consol)
pd_t

In [ ]:
ead_df_ = (mapPD_df_valid_test.merge(
    pd_t, how='left',
    left_on=['PD_POOL_ID', 'months'],
    right_on=['PD_POOL_ID', 'months']))

In [ ]:
ead_df_.to_clipboard()

In [ ]:
def calculate_PD_new(self,
                        ead_df: pd.DataFrame,
                        param: Dict[str, pd.DataFrame]) -> pd.DataFrame:

        pd_param_bond_mef = self._apply_mef(
            param_mef=param['MEFmultipliers'], pd_param_bond=param['PD_term_structure'])
        pd_param_bond_mef_monthly = self._convert_bond_pd_param_monthly(pd_param_bond_mef=pd_param_bond_mef)
        pd_param_consol = self._consol_pd_param(
            param_bond=pd_param_bond_mef_monthly, param=param)
        pd_t = self._transpose_pd(pd_param=pd_param_consol)


        ead_df_ = (ead_df.merge(
            pd_t, how='left',
            left_on=['PD_POOL_ID', 'months'],
            right_on=['PD_POOL_ID', 'months']))

        # 20250213 yj: as the mar_PD is monthly PD already, depreciated the calculation of monthly PD 
        
        ead_df_ = (ead_df_.assign(
            marPD_12m_freq=ead_df_['mar_PD'],
            marPD_lt_freq=ead_df_['mar_PD']))


        return ead_df_

In [ ]:
# Collective assessment (Valid cash flow route)
# EAD model
if ca_df_valid.shape[0] > 0:
    ecl_df_valid, ecl_interim_df_valid = (ca_df_valid
                                        .pipe(ce._merge_repayment_schedule,
                                                repay_df=repay_df)
                                        .pipe(ce._calculate_days)
                                        .pipe(ce._adjust_prepayment, param=param)
                                        .pipe(ce._adjust_ead_valid)
                                        .pipe(ce.calculate_PD, param=param)
                                        .pipe(ce.calcualte_LGD, param=param)
                                        .pipe(ce.calculate_discount_factor)
                                        .pipe(ce.calculate_engine_ecl,
                                                instr_df=ca_df_valid)
                                        )
else:
    ecl_df_valid = pd.DataFrame()
    ecl_interim_df_valid = pd.DataFrame()

In [ ]:
# # Collective assessment (Valid cash flow route)
# # EAD model
# if ca_df_valid.shape[0] > 0:
#     ecl_df_valid_old, ecl_interim_df_valid_old = (ca_df_valid
#                                         .pipe(ce._merge_repayment_schedule,
#                                                 repay_df=repay_df)
#                                         .pipe(ce._calculate_days)
#                                         .pipe(ce._adjust_prepayment_old, param=param)
#                                         .pipe(ce.calculate_PD, param=param)
#                                         .pipe(ce.calcualte_LGD, param=param)
#                                         .pipe(ce.calculate_discount_factor)
#                                         .pipe(ce.calculate_engine_ecl,
#                                                 instr_df=ca_df_valid)
#                                         )
# else:
#     ecl_df_valid_old = pd.DataFrame()
#     ecl_interim_df_valid_old = pd.DataFrame()

In [ ]:
# # Invalid cash flow route
# # 241112: we seperate on-bal and off-bal calculation due to duplicate keys in credit cards and credit lines
# # 1. ON
ca_df_invalid_on = ca_df_invalid.query(
    "ON_OFF_BAL_IND =='ON' and DRAWN_BAL_OCY > 0 and STAGE_FINAL < 3")

if ca_df_invalid_on.shape[0] > 0:
    repay_df_on_invalid = ce.repay_df_creation_bycondition(
        ca_df_invalid_on)

    ecl_df_invalid_on, ecl_df_interim_invalid_on = (ca_df_invalid_on
                                                    .pipe(ce._merge_repayment_schedule, repay_df=repay_df_on_invalid)
                                                    .pipe(ce._calculate_days)
                                                    .pipe(ce._adjust_prepayment, param=param)
                                                    .pipe(ce.calculate_PD, param=param)
                                                    .pipe(ce.calcualte_LGD, param=param)
                                                    .pipe(ce.calculate_discount_factor)
                                                    .pipe(ce.calculate_engine_ecl, instr_df=ca_df_invalid_on))
else:
    ecl_df_invalid_on = pd.DataFrame()
    ecl_df_interim_invalid_on = pd.DataFrame()


In [ ]:
# # 2. OFF
ca_df_invalid_off = ca_df_invalid.query(
    "ON_OFF_BAL_IND == 'OFF' and UNDRAWN_BAL_OCY > 0 and STAGE_FINAL < 3")
if ca_df_invalid_off.shape[0] > 0:
    repay_df_off_invalid = ce.repay_df_creation_bycondition(
        ca_df_invalid_off)

    ecl_df_invalid_off, ecl_df_interim_invalid_off = (ca_df_invalid_off
                                                    .pipe(ce._merge_repayment_schedule, repay_df=repay_df_off_invalid)
                                                    .pipe(ce._calculate_days)
                                                    .pipe(ce._adjust_prepayment, param=param)
                                                    .pipe(ce.calculate_PD, param=param)
                                                    .pipe(ce.calcualte_LGD, param=param)
                                                    .pipe(ce.calculate_discount_factor)
                                                    .pipe(ce.calculate_engine_ecl, instr_df=ca_df_invalid_off))
else:
    ecl_df_invalid_off = pd.DataFrame()
    ecl_df_interim_invalid_off = pd.DataFrame()

In [ ]:
# Below are stage 3:
if ca_nocf_input.shape[0] > 0:
    ca_nocf_df_1 = (ca_nocf_input.assign(years = str(1))) # as LGD does not hv term structure, set 1 to all
    ca_nocf_df_2 = ce.calcualte_LGD(ca_nocf_df_1,param=param)
    ca_nocf_df_2.LGD = ca_nocf_df_2.LGD.fillna(1) # as there are few records out of the Month in default's boundary, their LGD are all set to 1
    ecl_nocf_df, ecl_interim_nocf_df = ce._calculate_engine_stage3_ecl(ecl_df=ca_nocf_df_2,instr_df=ca_nocf_input)
else:
    ecl_nocf_df = pd.DataFrame()
    ecl_interim_nocf_df = pd.DataFrame()

In [ ]:
# Aggregate as a whole ecl result table
ecl_df = []
ecl_interim_df = []
ecl_df = pd.concat([ecl_df_valid, ecl_df_invalid_on,
                    ecl_df_invalid_off,
                    ecl_nocf_df]).reset_index(drop=True)

ecl_interim_df = pd.concat(
    [ecl_interim_df_valid, 
        ecl_df_interim_invalid_on, 
        ecl_df_interim_invalid_off,
        ecl_interim_nocf_df]).reset_index(drop=True)

In [ ]:
# # Invalid cash flow route
# # ON
# old approach

if ca_df_invalid_on.shape[0] > 0: #
    ecl_df_invalid_on_old, ecl_df_interim_invalid_on_old = (ca_df_invalid_on
                                                    .pipe(ce._merge_repayment_schedule, repay_df=repay_df_on_invalid)
                                                    .pipe(ce._calculate_days)
                                                    .pipe(ce._adjust_prepayment_old, param=param)
                                                    .pipe(ce.calculate_PD, param=param)
                                                    .pipe(ce.calcualte_LGD, param=param)
                                                    .pipe(ce.calculate_discount_factor)
                                                    .pipe(ce.calculate_engine_ecl, instr_df=ca_df_invalid_on))
else:
    ecl_df_invalid_on_old = pd.DataFrame()
    ecl_df_interim_invalid_on_old = pd.DataFrame()


In [ ]:
# Below are stage 3:
if ca_nocf_input.shape[0] > 0:
    ca_nocf_df_1 = (ca_nocf_input.assign(years = str(1))) # as LGD does not hv term structure, set 1 to all
    ca_nocf_df_2 = ce.calcualte_LGD(ca_nocf_df_1,param=param)
    ca_nocf_df_2.LGD = ca_nocf_df_2.LGD.fillna(1) # as there are few records out of the Month in default's boundary, their LGD are all set to 1
    ecl_nocf_df, ecl_interim_nocf_df = ce._calculate_engine_stage3_ecl(ecl_df=ca_nocf_df_2,instr_df=ca_nocf_input)
else:
    ecl_nocf_df = pd.DataFrame()
    ecl_interim_nocf_df = pd.DataFrame()

In [ ]:
ca_nocf_df_2.query("CONTRACT_ID == '248413'")

In [ ]:
# test for stage 3 correction

In [ ]:
def _calculate_engine_stage3_ecl(
                                    ecl_df: pd.DataFrame,
                                instr_df: pd.DataFrame
                                    ) -> pd.DataFrame:
    """
    To calculate the engine ECL amount for stage 3
    """
    #TODO: remove discount
    ecl_df_1_ = (ecl_df.assign(
        
        # For Model validation purpose
        IFRS9_PD_12M=1,

        IFRS9_PD_12M_MADJ=1, #TODO: stage 3 IFRS9_PD_12M_MADJ shall be 1 at first record

        IFRS9_PD_LT=1,

        IFRS9_LGD_12M=(ecl_df.LGD),

        IFRS9_LGD_LT=(ecl_df.LGD),


        EAD_PPADJ_12M=(ecl_df.EAD_OCY),
        EAD_PPADJ_LT=(ecl_df.EAD_OCY),
    ))

    ecl_df_ = (ecl_df_1_.assign(
        ECL_ENGINE_12M_OCY=(ecl_df_1_.EAD_PPADJ_12M
                            * ecl_df_1_.LGD),

        ECL_ENGINE_LT_OCY=(ecl_df_1_.EAD_PPADJ_LT
                            * ecl_df_1_.LGD),
    ))

    ecl_df_agg = (ecl_df_.groupby(by=['CONTRACT_ID'])
                    .agg(
        IFRS9_PD_12M=pd.NamedAgg('IFRS9_PD_12M', 'max'),
        IFRS9_PD_12M_MADJ=pd.NamedAgg(
            'IFRS9_PD_12M_MADJ', 'sum'),
        IFRS9_PD_LT=pd.NamedAgg('IFRS9_PD_LT', 'sum'),

        IFRS9_LGD_12M=pd.NamedAgg('IFRS9_LGD_12M', 'max'),
        IFRS9_LGD_LT=pd.NamedAgg('IFRS9_LGD_LT', 'mean'),

        ECL_ENGINE_12M_OCY=pd.NamedAgg(
            'ECL_ENGINE_12M_OCY', 'sum'),
        ECL_ENGINE_LT_OCY=pd.NamedAgg('ECL_ENGINE_LT_OCY', 'sum'),)
        .reset_index())

    ecl_final = (instr_df.merge(
        ecl_df_agg, how='left',
        left_on=['CONTRACT_ID'],
        right_on=['CONTRACT_ID']
    ))

    # To align with stage 1&2 data storage way
    cond = [
        ecl_final.STAGE_FINAL < 3,
        ecl_final.STAGE_FINAL == 3,
    ]

    choices = [
        ecl_final.ECL_ENGINE_12M_OCY,
        ecl_final.ECL_ENGINE_LT_OCY,
    ]

    ecl_final['ECL_ENGINE_OCY'] = (np.select(cond,
                                                choices,
                                                default=-999))

    return ecl_final, ecl_df_

In [ ]:
def _calculate_engine_stage3_ecl(
                                    ecl_df: pd.DataFrame,
                                instr_df: pd.DataFrame
                                    ) -> pd.DataFrame:
    """
    To calculate the engine ECL amount for stage 3
    """
    #TODO: remove discount
    ecl_df_1_ = (ecl_df.assign(
        
        # For Model validation purpose
        IFRS9_PD_12M=1,

        IFRS9_PD_12M_MADJ=1, #TODO: stage 3 IFRS9_PD_12M_MADJ shall be 1 at first record

        IFRS9_PD_LT=1,

        IFRS9_LGD_12M=(ecl_df.LGD),

        IFRS9_LGD_LT=(ecl_df.LGD),


        EAD_PPADJ_12M=(ecl_df.EAD_OCY),
        EAD_PPADJ_LT=(ecl_df.EAD_OCY),
    ))

    ecl_df_ = (ecl_df_1_.assign(
        ECL_ENGINE_12M_OCY=(ecl_df_1_.EAD_PPADJ_12M
                            * ecl_df_1_.LGD),

        ECL_ENGINE_LT_OCY=(ecl_df_1_.EAD_PPADJ_LT
                            * ecl_df_1_.LGD),
    ))

    ecl_df_agg = (ecl_df_.groupby(by=['CONTRACT_ID', 'ON_OFF_BAL_IND'])
                    .agg(
        IFRS9_PD_12M=pd.NamedAgg('IFRS9_PD_12M', 'max'),
        IFRS9_PD_12M_MADJ=pd.NamedAgg(
            'IFRS9_PD_12M_MADJ', 'sum'),
        IFRS9_PD_LT=pd.NamedAgg('IFRS9_PD_LT', 'sum'),

        IFRS9_LGD_12M=pd.NamedAgg('IFRS9_LGD_12M', 'max'),
        IFRS9_LGD_LT=pd.NamedAgg('IFRS9_LGD_LT', 'mean'),

        ECL_ENGINE_12M_OCY=pd.NamedAgg(
            'ECL_ENGINE_12M_OCY', 'sum'),
        ECL_ENGINE_LT_OCY=pd.NamedAgg('ECL_ENGINE_LT_OCY', 'sum'),)
        .reset_index())

    ecl_final = (instr_df.merge(
        ecl_df_agg, how='left',
        left_on=['CONTRACT_ID', 'ON_OFF_BAL_IND'],
        right_on=['CONTRACT_ID', 'ON_OFF_BAL_IND']
    ))

    # To align with stage 1&2 data storage way
    cond = [
        ecl_final.STAGE_FINAL < 3,
        ecl_final.STAGE_FINAL == 3,
    ]

    choices = [
        ecl_final.ECL_ENGINE_12M_OCY,
        ecl_final.ECL_ENGINE_LT_OCY,
    ]

    ecl_final['ECL_ENGINE_OCY'] = (np.select(cond,
                                                choices,
                                                default=-999))

    return ecl_final, ecl_df_

In [ ]:
ca_nocf_df_1 = (ca_nocf_input.assign(years = str(1))) # as LGD does not hv term structure, set 1 to all
ca_nocf_df_2 = ce.calcualte_LGD(ca_nocf_df_1,param=param)
ca_nocf_df_2.LGD = ca_nocf_df_2.LGD.fillna(1) # as there are few records out of the Month in default's boundary, their LGD are all set to 1

In [ ]:
ecl_nocf_df_test_tgt, ecl_interim_nocf_df_test_tgt = _calculate_engine_stage3_ecl(ecl_df=ca_nocf_df_2,instr_df=ca_nocf_input)

In [ ]:
ecl_nocf_df_test_tgt = ecl_nocf_df_test_tgt.assign(ecl_per = ecl_nocf_df_test_tgt.ECL_ENGINE_OCY/ecl_nocf_df_test_tgt.EAD_OCY)

In [ ]:
ecl_nocf_df_test_tgt.query("CONTRACT_ID == '248413'").to_clipboard()

In [ ]:
ecl_nocf_df_test_tgt.query("ecl_per>1")